# **Question 3. Class hierarchy**


# **3a**

In [2]:
import os
import json


class ClassHierarchy:
    def __init__(self, filename, filename2):
        with open(filename, "r") as f:
            lines = f.readlines()

        self.hierarchy = {}
        for line in lines:
            parent, child = line.strip().split()
            self.hierarchy[child] = parent


        with open(filename2, "r") as f:
            lines = f.readlines()

        self.hierarchy_id_to_name = {}
        for line in lines:
            char = line.strip().replace(', ', '_').replace('\t', '_').replace(',', '').split("_")
            class_id, class_name = char[:1], char[1:]
            self.hierarchy_id_to_name[class_id[0]] = []
            for x in class_name:
                # print(class_id, x)
                self.hierarchy_id_to_name[class_id[0]].append(x)

    def find_siblings(self, class_name):
        siblings = []
        parent = self.hierarchy[class_name]
        for child in self.hierarchy:
            if self.hierarchy[child] == parent:
                siblings.append(child)

        results = []
        for i in siblings:
            results.append(self.hierarchy_id_to_name[i])
        return results

    def find_parent(self, class_name):

        return self.hierarchy_id_to_name[str(self.hierarchy[class_name])]

    def find_ancestors(self, class_name):
        ancestors = []
        parent = self.hierarchy[class_name]
        while parent:
            ancestors.append(parent)
            # print(parent)
            if parent in self.hierarchy.keys():
                parent = self.hierarchy[parent]
            else:
              break
        results = []
        for i in ancestors:
            results.append(self.hierarchy_id_to_name[i])
        return results


    def find_if_same_ancestor(self, class_1, class_2):
        ancestors_1 = self.find_ancestors(class_1)
        ancestors_2 = self.find_ancestors(class_2)

        for ancestor in ancestors_1:
            if ancestor in ancestors_2:
                return True

        return False


if __name__ == "__main__":
    hierarchy = ClassHierarchy("hierarchy.txt", 'id_to_name.txt')

    print("Siblings of", hierarchy.hierarchy_id_to_name['n02431122'])
    print(hierarchy.find_siblings("n02431122"))
    print("Parent of", hierarchy.hierarchy_id_to_name['n02431122'])
    print( hierarchy.find_parent("n02431122"))
    print("Ancestors", hierarchy.hierarchy_id_to_name['n02431122'])
    print(hierarchy.find_ancestors("n02431122"))
    print("Do", hierarchy.hierarchy_id_to_name['n02431122']," and ", hierarchy.hierarchy_id_to_name['n02418064'], "have the same ancestor?:")
    print( hierarchy.find_if_same_ancestor("n02431122", "n02418064"))

Siblings of ['red deer', 'elk', 'American elk', 'wapiti', 'Cervus elaphus']
[['Japanese deer', 'sika', 'Cervus nipon', 'Cervus sika'], ['musk deer', 'Moschus moschiferus'], ['mule deer', 'burro deer', 'Odocoileus hemionus'], ["pere david's deer", 'elaphure', 'Elaphurus davidianus'], ['brocket'], ['red deer', 'elk', 'American elk', 'wapiti', 'Cervus elaphus'], ['Virginia deer', 'white tail', 'whitetail', 'white-tailed deer', 'whitetail deer', 'Odocoileus Virginianus'], ['fallow deer', 'Dama dama'], ['caribou', 'reindeer', 'Greenland caribou', 'Rangifer tarandus'], ['sambar', 'sambur', 'Cervus unicolor'], ['pricket'], ['elk', 'European elk', 'moose', 'Alces alces'], ['wapiti', 'elk', 'American elk', 'Cervus elaphus canadensis'], ['muntjac', 'barking deer'], ['roe deer', 'Capreolus capreolus']]
Parent of ['red deer', 'elk', 'American elk', 'wapiti', 'Cervus elaphus']
['deer', 'cervid']
Ancestors ['red deer', 'elk', 'American elk', 'wapiti', 'Cervus elaphus']
[['deer', 'cervid'], ['ruminan

# **3c**

In [ ]:
def nms(boxes, scores, classes, class_hierarchy, score_threshold, iou_threshold):
    keep = []
    for box, score, class_name in zip(boxes, scores, classes):
        if score < score_threshold:
            continue
        keep.append((box, score, class_name))

    while keep:
        box, score, class_name = keep.pop(0)
        for box2, score2, class_name2 in keep:
            if class_name == class_name2:
                if iou(box, box2) > iou_threshold:
                    if score2 < score:
                        keep.remove((box2, score2, class_name2))
            else:
                if class_hierarchy[class_name2] == class_name:
                    keep.remove((box2, score2, class_name2))

    return keep

def iou(box1, box2):
    """Computes the intersection over union of two bounding boxes."""
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2
    width_overlap = min(x2, x4) - max(x1, x3)
    height_overlap = min(y2, y4) - max(y1, y3)
    if width_overlap <= 0 or height_overlap <= 0:
        return 0.0
    area_overlap = width_overlap * height_overlap
    area1 = (x2 - x1) * (y2 - y1)
    area2 = (x4 - x3) * (y4 - y3)
    iou = area_overlap / (area1 + area2 - area_overlap)
    return iou